<a href="https://colab.research.google.com/github/Daniel-Fernandez-951/build-week/blob/master/Build-1/Daniel_Fernandez_DS20_BuildWeek_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [261]:
# Data Handling
import pandas as pd
import numpy as np
# Libraries
import plotly.express as px

In [ ]:
!wget "https://healthdata.gov/node/3376186/download" -O detailed_covid.csv

In [157]:
# Set columns dtype
col_dtyp = {'current_status': str, 'sex': str, 'age_group': str,
 'Race and ethnicity (combined)': str, 'hosp_yn': str, 'icu_yn': str, 'death_yn': str,
 'medcond_yn': str}

# data = pd.read_csv("/content/detailed_covid.csv", dtype=col_dtyp)
data = pd.read_csv("/content/detailed_covid.csv")
data = data.drop(['pos_spec_dt', 'onset_dt', 'Race and ethnicity (combined)'], axis=1)
# # list(data.columns)
data.head(20)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,cdc_report_dt,current_status,sex,age_group,hosp_yn,icu_yn,death_yn,medcond_yn
0,2020/03/03,Laboratory-confirmed case,Male,0 - 9 Years,Missing,Missing,Missing,Missing
1,2020/03/03,Laboratory-confirmed case,Female,0 - 9 Years,Missing,Missing,Missing,Missing
2,2020/04/07,Laboratory-confirmed case,Unknown,0 - 9 Years,No,Missing,Missing,Missing
3,2020/08/04,Probable Case,Male,0 - 9 Years,Missing,Missing,Missing,Missing
4,2020/07/28,Laboratory-confirmed case,Male,0 - 9 Years,No,No,No,Missing
5,2020/08/03,Laboratory-confirmed case,Male,0 - 9 Years,Missing,Missing,Missing,Missing
6,2020/08/04,Laboratory-confirmed case,Male,0 - 9 Years,Unknown,Unknown,No,Unknown
7,2020/08/04,Laboratory-confirmed case,Male,0 - 9 Years,Unknown,Unknown,No,Unknown
8,2020/08/04,Laboratory-confirmed case,Male,0 - 9 Years,Unknown,Unknown,No,Unknown
9,2020/08/04,Laboratory-confirmed case,Male,0 - 9 Years,Missing,Missing,Missing,Missing


In [165]:
data['death_01'] = data['death_yn'].map({'Yes': 1, 'No': 0})
# data.isna().sum() # 1892053
print(f'Data available after removal = {data.shape[0] - 1892053}')
print(f'Percentage of data removed = {round(((data.shape[0]-1892053)/data.shape[0]*100),2)}%')

Data available after removal = 1770272
Percentage of data removed = 48.34%


1892053 rows had no information (unknown or missing) for `death_yn`, after binary replacement of "Yes" and "No", drop all 1892053 missing/unknown values in `death_01` column.

In [166]:
# Drop NaN
data_clean = data.dropna(how='any', subset=['death_01'])

In [167]:
# Combine `Unknown` and `NaN` to `Unknown Age`
data_clean['age_grp'] = data['age_group'].replace(regex=['Unknown', np.nan], value='Unknown Age')
data_clean['age_grp'].value_counts(dropna=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


20 - 29 Years    321678
30 - 39 Years    289986
40 - 49 Years    270011
50 - 59 Years    263159
60 - 69 Years    192350
10 - 19 Years    133292
80+ Years        123121
70 - 79 Years    118866
0 - 9 Years       57170
Unknown Age         639
Name: age_grp, dtype: int64

In [168]:
# Convert `Missing` to Unknown
data_clean['medcond_clean'] = data['medcond_yn'].replace(regex=['Missing', 'Unknown'], value='Unknown')
data_clean['medcond_clean'].value_counts(dropna=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Unknown    1213238
Yes         325174
No          231860
Name: medcond_clean, dtype: int64

In [220]:
data_table = pd.DataFrame(data_clean[['age_grp', 'medcond_clean', 'death_01']])
data_table

,age_grp,medcond_clean,death_01
4,0 - 9 Years,Unknown,0.0
6,0 - 9 Years,Unknown,0.0
7,0 - 9 Years,Unknown,0.0
8,0 - 9 Years,Unknown,0.0
11,0 - 9 Years,Unknown,0.0
...,...,...,...
3662320,30 - 39 Years,Yes,0.0
3662321,30 - 39 Years,Unknown,0.0
3662322,30 - 39 Years,Unknown,0.0
3662323,30 - 39 Years,Unknown,0.0


In [228]:
# Pivot table
table = data_clean.pivot_table(columns='age_grp', index=['death_yn', 'medcond_clean'], values='death_01', aggfunc='count' )
table

age_grp                 0 - 9 Years  10 - 19 Years  ...  80+ Years  Unknown Age
death_yn medcond_clean                              ...                        
No       No                   10467          26768  ...       1867           78
         Unknown              42353          94462  ...      45982          477
         Yes                   4306          11990  ...      18088           64
Yes      No                       8              4  ...        679            1
         Unknown                 25             39  ...      29029           11
         Yes                     11             29  ...      27476            8

[6 rows x 10 columns]

In [ ]:
# Test conditionals on data
# test = ((data_clean['medcond_yn'] == 'Yes') & (data_clean['death_01'] == 1))
# data_clean[test].count()

In [ ]:
# https://www.cdc.gov/nchs/nvss/vsrr/covid_weekly/index.htm#Comorbidities

In [270]:

fig = px.bar(data_clean, x='age_grp', y='count')
fig.show()

ValueError: ignored